In [1]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

In [2]:
APP_KEY = "Access Key"
APP_SECRET = "Secret Key"
URL_BASE = "https://openapivts.koreainvestment.com:29443" # 모의투자
ACCESS_TOKEN = ""

In [3]:
def auth(): #토큰 발급
    headers = {"content-type":"application/json"}
    body = {
        "grant_type":"client_credentials",
        "appkey":APP_KEY, 
        "appsecret":APP_SECRET
        }
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    
    global ACCESS_TOKEN
    ACCESS_TOKEN = res.json()["access_token"]

In [4]:
auth()

In [5]:
# 조회 데이터 
def itemchartprice(stock_no, start:str, end:str): #일봉 데이터 조회(한번 호출시 마다 최대 100건까지만 조회 가능)
    PATH = "uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
    URL = f"{URL_BASE}/{PATH}"

    # 헤더 설정
    headers = {"Content-Type":"application/json", 
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey":APP_KEY,
               "appSecret":APP_SECRET,
               "tr_id":"FHKST03010100"}
    params = {
            "fid_cond_mrkt_div_code": "J",
            "fid_input_date_1": start,
            "fid_input_date_2": end,
            "fid_input_iscd": stock_no,
            "fid_org_adj_prc": "0",
            "fid_period_div_code": "D"
    }
    
    # 호출
    res = requests.get(URL, headers=headers, params=params)
    data = pd.DataFrame(res.json()["output2"])


    end = datetime.strftime(datetime.strptime(data.iloc[-1]["stck_bsop_date"],'%Y%m%d') - timedelta(days=1),'%Y%m%d')
    while  end > start:
        params['fid_input_date_2'] = end
        last_date = data.iloc[-1]["stck_bsop_date"]
        res = requests.get(URL, headers=headers, params=params)
        if res.json()['output2'][0] == {}:
            break
        # data = data.append(pd.DataFrame(res.json()["output2"]))
        data = pd.concat([data, pd.DataFrame(res.json()["output2"])])

        if last_date == data.iloc[-1]["stck_bsop_date"]: 
            break
        end = datetime.strftime(datetime.strptime(data.iloc[-1]["stck_bsop_date"],'%Y%m%d') - timedelta(days=1),'%Y%m%d')
    
    #Parsing
    values =  ["Date", "Open", "High", "Low", "Close", "Volume"]
    keys = ["stck_bsop_date", "stck_oprc", "stck_hgpr", "stck_lwpr", "stck_clpr", "acml_vol"]
    data = data[keys]
    rename_map = dict(zip(keys, values))
    data.rename(columns=rename_map, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d', errors='raise')
    data = data.set_index("Date")
    data.astype('int')
    data = data.astype("int64")
    return(data)  

In [7]:
data = itemchartprice("005930",'20010201','20201231')
data.to_csv('data.csv')
data.head()


,Open,High,Low,Close,Volume
Date,,,,,
2020-12-30,77400,81300,77300,81000,29417420
2020-12-29,78800,78900,77300,78300,30339448
2020-12-28,79000,80100,78200,78700,40085044
2020-12-24,74100,78800,74000,77800,32502870
2020-12-23,72400,74000,72300,73900,19411326
